In [0]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!mkdir -p ~/.kaggle



import json 

token = {"username":"seunghwan1228","key":"2a8fb35ecd319bf10a02713c306059ca"}

with open('/content/drive/My Drive/Colab Notebooks/Kaggle Kernel/kaggle.json', 'w') as file:
  json.dump(token, file)
  
  
  
!cp drive/'My Drive'/'Colab Notebooks'/'Kaggle Kernel'/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle
!ls -l ~/.kaggle
!cat ~/.kaggle/kaggle.json


# down load path

!kaggle competitions download -c jigsaw-unintended-bias-in-toxicity-classification

kaggle.json
total 4
-rw------- 1 root root 72 Sep 16 18:36 kaggle.json
{"username": "seunghwan1228", "key": "2a8fb35ecd319bf10a02713c306059ca"}Downloading train.csv.zip to /content
 93% 255M/273M [00:01<00:00, 173MB/s]
100% 273M/273M [00:02<00:00, 142MB/s]
 42% 5.00M/12.0M [00:00<00:00, 32.8MB/s]
100% 12.0M/12.0M [00:00<00:00, 58.3MB/s]
  0% 0.00/224k [00:00<?, ?B/s]
100% 224k/224k [00:00<00:00, 144MB/s]


In [0]:
!kaggle datasets list -s 

usage: kaggle datasets list [-h] [--sort-by SORT_BY] [--size SIZE]
                            [--file-type FILE_TYPE] [--license LICENSE_NAME]
                            [--tags TAG_IDS] [-s SEARCH] [-m] [--user USER]
                            [-p PAGE] [-v]
kaggle datasets list: error: argument -s/--search: expected one argument


In [0]:
!kaggle datasets files zillow/zecon

name                           size  creationDate         
----------------------------  -----  -------------------  
all_available_metrics.json      3KB  2018-01-24 23:45:58  
cities_crosswalk.csv            1MB  2018-01-24 23:54:02  
City_time_series.csv          658MB  2018-01-24 23:46:25  
County_time_series.csv        108MB  2018-01-24 23:46:05  
CountyCrossWalk_Zillow.csv    227KB  2018-01-24 23:54:02  
DataDictionary.csv              5KB  2018-01-24 23:54:02  
fields_per_level.json          17KB  2018-01-24 23:45:58  
Metro_time_series.csv          54MB  2018-01-24 23:46:02  
Neighborhood_time_series.csv  253MB  2018-01-24 23:46:11  
State_time_series.csv           5MB  2018-01-24 23:53:14  
Zip_time_series.csv           746MB  2018-01-24 23:46:23  


In [0]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip

--2019-09-16 18:36:15--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.20.22.166, 104.20.6.166, 2606:4700:10::6814:16a6, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.20.22.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1523785255 (1.4G) [application/zip]
Saving to: ‘crawl-300d-2M.vec.zip’

crawl-300d-2M.vec.z 100%[===================>]   1.42G  29.3MB/s    in 50s     

2019-09-16 18:37:05 (29.2 MB/s) - ‘crawl-300d-2M.vec.zip’ saved [1523785255/1523785255]



In [0]:
!unzip /content/crawl-300d-2M.vec.zip

Archive:  /content/crawl-300d-2M.vec.zip
  inflating: crawl-300d-2M.vec       


In [0]:
# !wget http://nlp.stanford.edu/data/glove.840B.300d.zip

In [0]:
!unzip train.csv.zip
!unzip test.csv.zip

Archive:  train.csv.zip
  inflating: train.csv               
Archive:  test.csv.zip
  inflating: test.csv                


In [0]:
import gc
import re
import operator 

import numpy as np
import pandas as pd

from gensim.models import KeyedVectors

from sklearn import model_selection

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, Input, Dense, CuDNNGRU,concatenate, Bidirectional, SpatialDropout1D, Conv1D, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.optimizers import RMSprop, Adam
from keras.models import Model
from keras.callbacks import EarlyStopping

import seaborn as sns

Using TensorFlow backend.


In [0]:
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')

In [0]:
train.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,black,buddhist,christian,female,heterosexual,hindu,homosexual_gay_or_lesbian,intellectual_or_learning_disability,jewish,latino,male,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:41.987077+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:42.870083+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:45.222647+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:47.601894+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-29 10:50:48.488476+00,2,NaN,2006,rejected,0,0,0,1,0,0.0,4,47


In [0]:
test.head()

,id,comment_text
0,7097320,[ Integrity means that you pay your debts.]\n\...
1,7097321,This is malfeasance by the Administrator and t...
2,7097322,@Rmiller101 - Spoken like a true elitist. But ...
3,7097323,"Paul: Thank you for your kind words. I do, in..."
4,7097324,Sorry you missed high school. Eisenhower sent ...


# For preprocessing  concat train and test : axis = 0

In [0]:
df = pd.concat([train[['id', 'comment_text']], test], axis = 0)

In [0]:
ft_common_crawl = '/content/crawl-300d-2M.vec'

# load .vec files 
embedding_index = KeyedVectors.load_word2vec_format(ft_common_crawl)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
embedding_index.vectors

(1999995, 300)


array([[-0.0282, -0.0557, -0.0451, ..., -0.037 , -0.0725, -0.0042],
       [ 0.0231,  0.017 ,  0.0157, ...,  0.0744, -0.1118,  0.0963],
       [-0.0849, -0.0582, -0.0321, ...,  0.0032, -0.0237, -0.0366],
       ...,
       [-0.022 , -0.5017,  0.2547, ..., -0.2723, -0.4114,  0.0499],
       [ 0.1029, -0.0357,  0.2598, ...,  0.0879,  0.1195, -0.3743],
       [ 0.5513,  0.0287,  0.1653, ..., -0.1665, -0.0714,  0.3548]],
      dtype=float32)

In [0]:
embedding_index.vocab

{',': <gensim.models.keyedvectors.Vocab at 0x7feba07060b8>,
 'the': <gensim.models.keyedvectors.Vocab at 0x7feba0743400>,
 '.': <gensim.models.keyedvectors.Vocab at 0x7feba0706128>,
 'and': <gensim.models.keyedvectors.Vocab at 0x7febffe697b8>,
 'to': <gensim.models.keyedvectors.Vocab at 0x7feba0706198>,
 'of': <gensim.models.keyedvectors.Vocab at 0x7febffe69208>,
 'a': <gensim.models.keyedvectors.Vocab at 0x7feba0706208>,
 'in': <gensim.models.keyedvectors.Vocab at 0x7febffe69160>,
 'is': <gensim.models.keyedvectors.Vocab at 0x7feba0706278>,
 'for': <gensim.models.keyedvectors.Vocab at 0x7feba06d80b8>,
 'that': <gensim.models.keyedvectors.Vocab at 0x7feba07062e8>,
 'I': <gensim.models.keyedvectors.Vocab at 0x7feba06d8278>,
 'it': <gensim.models.keyedvectors.Vocab at 0x7feba0706320>,
 'on': <gensim.models.keyedvectors.Vocab at 0x7feba0706048>,
 'with': <gensim.models.keyedvectors.Vocab at 0x7feba0706390>,
 ')': <gensim.models.keyedvectors.Vocab at 0x7feba068f3c8>,
 ':': <gensim.models.k

In [0]:
print(embedding_index.vectors.shape)  # 300dim embedding

(1999995, 300)


In [0]:
embedding_index.vocab

{',': <gensim.models.keyedvectors.Vocab at 0x7feba07060b8>,
 'the': <gensim.models.keyedvectors.Vocab at 0x7feba0743400>,
 '.': <gensim.models.keyedvectors.Vocab at 0x7feba0706128>,
 'and': <gensim.models.keyedvectors.Vocab at 0x7febffe697b8>,
 'to': <gensim.models.keyedvectors.Vocab at 0x7feba0706198>,
 'of': <gensim.models.keyedvectors.Vocab at 0x7febffe69208>,
 'a': <gensim.models.keyedvectors.Vocab at 0x7feba0706208>,
 'in': <gensim.models.keyedvectors.Vocab at 0x7febffe69160>,
 'is': <gensim.models.keyedvectors.Vocab at 0x7feba0706278>,
 'for': <gensim.models.keyedvectors.Vocab at 0x7feba06d80b8>,
 'that': <gensim.models.keyedvectors.Vocab at 0x7feba07062e8>,
 'I': <gensim.models.keyedvectors.Vocab at 0x7feba06d8278>,
 'it': <gensim.models.keyedvectors.Vocab at 0x7feba0706320>,
 'on': <gensim.models.keyedvectors.Vocab at 0x7feba0706048>,
 'with': <gensim.models.keyedvectors.Vocab at 0x7feba0706390>,
 ')': <gensim.models.keyedvectors.Vocab at 0x7feba068f3c8>,
 ':': <gensim.models.k

In [0]:
assert embedding_index.get_vector(',').all() == embedding_index.vectors[0].all()

# text preprocessing

As with most NLP tasks, we will start by using some pre-trained embeddings for our words. This provides us with a numerical representation of our input that we can use for modelling. Mapping words to embeddings isn't always straight forward, however: the data may not be very tidy.

The first step, then, is to ensure we get as many words mapped to a suitable embedding as possible. To do this, we'll make use of two excellent kernels:

## checker

In [0]:
# build voca index dictionary

# ex) texts = ['asd fbv', 'qwe rasdf']
# split : [[asd, fbc], [qwe, rasdf]]

def build_vocab(texts):
    sentences = texts.apply(lambda x: x.split()).values
    vocab = {}
    for sentence in sentences:
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab
  
  # return as {',' : n,
  #            '.' : m}

In [0]:
# compare voca dictionary and embedding keys

def check_coverage(vocab, embeddings_index):
    known_words = {}
    unknown_words = {}
    nb_known_words = 0
    nb_unknown_words = 0
    for word in vocab.keys(): # from build vocab 
        try:
            known_words[word] = embeddings_index[word]
            nb_known_words += vocab[word]
        except:
            unknown_words[word] = vocab[word]
            nb_unknown_words += vocab[word]
            pass

    print('Found embeddings for {:.3%} of vocab'.format(len(known_words) / len(vocab)))
    print('Found embeddings for  {:.3%} of all text'.format(nb_known_words / (nb_known_words + nb_unknown_words)))
    unknown_words = sorted(unknown_words.items(), key=operator.itemgetter(1))[::-1]

    return unknown_words

In [0]:
embedding_index[',']

array([-2.820e-02, -5.570e-02, -4.510e-02, -4.340e-02,  7.120e-02,
       -8.550e-02, -1.085e-01, -5.610e-02, -4.523e-01, -2.020e-02,
        9.750e-02,  1.047e-01,  1.962e-01, -6.930e-02,  2.130e-02,
       -2.350e-02,  1.336e-01, -4.200e-02, -5.640e-02, -7.980e-02,
        4.240e-02, -4.090e-02, -5.360e-02, -2.520e-02,  1.350e-02,
        6.400e-03,  1.235e-01,  4.610e-02,  1.200e-02, -3.720e-02,
        6.500e-02,  4.100e-03, -1.074e-01, -2.630e-02,  1.133e-01,
       -2.900e-03,  6.710e-02,  1.065e-01,  2.340e-02, -1.600e-02,
        7.000e-03,  4.355e-01, -7.520e-02, -4.328e-01,  4.570e-02,
        6.040e-02, -7.400e-02, -5.500e-03, -8.900e-03, -2.926e-01,
       -5.450e-02, -1.519e-01,  9.900e-02, -1.930e-02, -5.000e-03,
        5.110e-02,  4.040e-02,  1.023e-01, -1.280e-02,  4.880e-02,
       -1.567e-01, -7.590e-02, -1.900e-02,  1.442e-01,  4.700e-03,
       -1.860e-02,  1.400e-02, -3.850e-02, -8.530e-02,  1.572e-01,
        1.770e-01,  8.400e-03, -2.500e-02, -1.145e-01, -6.630e

## with concated df

In [0]:
df.head()

,id,comment_text
0,59848,"This is so cool. It's like, 'would you want yo..."
1,59849,Thank you!! This would make my life a lot less...
2,59852,This is such an urgent design problem; kudos t...
3,59855,Is this something I'll be able to install on m...
4,59856,haha you guys are a bunch of losers.


In [0]:
df['comment_text'] = df['comment_text'].apply(lambda x : x.lower())

gc.collect()

0

In [0]:
df.head()

,id,comment_text
0,59848,"this is so cool. it's like, 'would you want yo..."
1,59849,thank you!! this would make my life a lot less...
2,59852,this is such an urgent design problem; kudos t...
3,59855,is this something i'll be able to install on m...
4,59856,haha you guys are a bunch of losers.


In [0]:
# chekc first 

vocab = build_vocab(df['comment_text'])
oov = check_coverage(vocab, embedding_index)

Found embeddings for 12.868% of vocab
Found embeddings for  91.201% of all text


In [0]:
oov[:10]

[("i'm", 87857),
 ("i've", 28456),
 ('"the', 26420),
 ("trump's", 25818),
 ("let's", 24149),
 ("aren't", 22715),
 ("wouldn't", 22217),
 ('so,', 21352),
 ("wasn't", 20247),
 ("i'd", 19344)]

In [0]:
gc.collect()

129

In [0]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have" }

In [0]:
del(vocab,oov)
gc.collect()

0

In [0]:
def known_contractions(embed):
    known = []
    for contract in contraction_mapping:
        if contract in embed:
            known.append(contract)
    return known

In [0]:
print("- Known Contractions -")
print("   FastText :")
print(known_contractions(embedding_index))

- Known Contractions -
   FastText :
["can't", "'cause", "didn't", "doesn't", "don't", "haven't", "he's", "I'd", "I'll", "I'm", "I've", "isn't", "it's", "she's", "that's", "there's", "they're", "we're", "won't", "you'll", "you're"]


In [0]:
# convert special uppestopy's to " ' " 

def clean_contractions(text, mapping):
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(" ")])
    return text

In [1]:
df['comment_text'] = df['comment_text'].apply(lambda x: clean_contractions(x, contraction_mapping))

NameError: ignored

In [0]:
vocab = build_vocab(df['comment_text'])
oov = check_coverage(vocab, embedding_index)
oov[:10]

Found embeddings for 12.942% of vocab
Found embeddings for  91.709% of all text


[("trump's", 28554),
 ('"the', 26420),
 ('so,', 21352),
 ('no,', 17866),
 ('trump.', 17644),
 ('it?', 12989),
 ('trump,', 11868),
 ('but,', 10532),
 ('fact,', 10040),
 ('"i', 9922)]

In [0]:
del(vocab,oov)
gc.collect()

0

In [0]:
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'

In [0]:
# check what punct dont know the embedding index

def unknown_punct(embed, punct):
    unknown = ''
    for p in punct:
        if p not in embed:
            unknown += p
            unknown += ' '
    return unknown

In [0]:
print(unknown_punct(embedding_index, punct))

_ ` 


In [0]:
punct_mapping = {"_":" ", "`":" "}

In [0]:
def clean_special_chars(text, punct, mapping):
    for p in mapping:
        text = text.replace(p, mapping[p])    
    for p in punct:
        text = text.replace(p, f' {p} ')     
    return text

In [0]:
df['comment_text'] = df['comment_text'].apply(lambda x: clean_special_chars(x, punct, punct_mapping))

In [0]:
vocab = build_vocab(df['comment_text'])
oov = check_coverage(vocab, embedding_index)

Found embeddings for 47.176% of vocab
Found embeddings for  99.521% of all text


In [0]:
del(vocab,oov)
gc.collect()

0

In [0]:
# Let's replace any swear words we don't have an embedding for with something we do ;)
swear_words = [
    ' 4r5e ',
    ' 5h1t ',
    ' 5hit ',
    ' a55 ',
    ' anal ',
    ' anus ',
    ' ar5e ',
    ' arrse ',
    ' arse ',
    ' ass ',
    ' ass-fucker ',
    ' asses ',
    ' assfucker ',
    ' assfukka ',
    ' asshole ',
    ' assholes ',
    ' asswhole ',
    ' a_s_s ',
    ' b!tch ',
    ' b00bs ',
    ' b17ch ',
    ' b1tch ',
    ' ballbag ',
    ' balls ',
    ' ballsack ',
    ' bastard ',
    ' beastial ',
    ' beastiality ',
    ' bellend ',
    ' bestial ',
    ' bestiality ',
    ' biatch ',
    ' bitch ',
    ' bitcher ',
    ' bitchers ',
    ' bitches ',
    ' bitchin ',
    ' bitching ',
    ' bloody ',
    ' blow job ',
    ' blowjob ',
    ' blowjobs ',
    ' boiolas ',
    ' bollock ',
    ' bollok ',
    ' boner ',
    ' boob ',
    ' boobs ',
    ' booobs ',
    ' boooobs ',
    ' booooobs ',
    ' booooooobs ',
    ' breasts ',
    ' buceta ',
    ' bugger ',
    ' bum ',
    ' bunny fucker ',
    ' butt ',
    ' butthole ',
    ' buttmuch ',
    ' buttplug ',
    ' c0ck ',
    ' c0cksucker ',
    ' carpet muncher ',
    ' cawk ',
    ' chink ',
    ' cipa ',
    ' cl1t ',
    ' clit ',
    ' clitoris ',
    ' clits ',
    ' cnut ',
    ' cock ',
    ' cock-sucker ',
    ' cockface ',
    ' cockhead ',
    ' cockmunch ',
    ' cockmuncher ',
    ' cocks ',
    ' cocksuck ',
    ' cocksucked ',
    ' cocksucker ',
    ' cocksucking ',
    ' cocksucks ',
    ' cocksuka ',
    ' cocksukka ',
    ' cok ',
    ' cokmuncher ',
    ' coksucka ',
    ' coon ',
    ' cox ',
    ' crap ',
    ' cum ',
    ' cummer ',
    ' cumming ',
    ' cums ',
    ' cumshot ',
    ' cunilingus ',
    ' cunillingus ',
    ' cunnilingus ',
    ' cunt ',
    ' cuntlick ',
    ' cuntlicker ',
    ' cuntlicking ',
    ' cunts ',
    ' cyalis ',
    ' cyberfuc ',
    ' cyberfuck ',
    ' cyberfucked ',
    ' cyberfucker ',
    ' cyberfuckers ',
    ' cyberfucking ',
    ' d1ck ',
    ' damn ',
    ' dick ',
    ' dickhead ',
    ' dildo ',
    ' dildos ',
    ' dink ',
    ' dinks ',
    ' dirsa ',
    ' dlck ',
    ' dog-fucker ',
    ' doggin ',
    ' dogging ',
    ' donkeyribber ',
    ' doosh ',
    ' duche ',
    ' dyke ',
    ' ejaculate ',
    ' ejaculated ',
    ' ejaculates ',
    ' ejaculating ',
    ' ejaculatings ',
    ' ejaculation ',
    ' ejakulate ',
    ' f u c k ',
    ' f u c k e r ',
    ' f4nny ',
    ' fag ',
    ' fagging ',
    ' faggitt ',
    ' faggot ',
    ' faggs ',
    ' fagot ',
    ' fagots ',
    ' fags ',
    ' fanny ',
    ' fannyflaps ',
    ' fannyfucker ',
    ' fanyy ',
    ' fatass ',
    ' fcuk ',
    ' fcuker ',
    ' fcuking ',
    ' feck ',
    ' fecker ',
    ' felching ',
    ' fellate ',
    ' fellatio ',
    ' fingerfuck ',
    ' fingerfucked ',
    ' fingerfucker ',
    ' fingerfuckers ',
    ' fingerfucking ',
    ' fingerfucks ',
    ' fistfuck ',
    ' fistfucked ',
    ' fistfucker ',
    ' fistfuckers ',
    ' fistfucking ',
    ' fistfuckings ',
    ' fistfucks ',
    ' flange ',
    ' fook ',
    ' fooker ',
    ' fuck ',
    ' fucka ',
    ' fucked ',
    ' fucker ',
    ' fuckers ',
    ' fuckhead ',
    ' fuckheads ',
    ' fuckin ',
    ' fucking ',
    ' fuckings ',
    ' fuckingshitmotherfucker ',
    ' fuckme ',
    ' fucks ',
    ' fuckwhit ',
    ' fuckwit ',
    ' fudge packer ',
    ' fudgepacker ',
    ' fuk ',
    ' fuker ',
    ' fukker ',
    ' fukkin ',
    ' fuks ',
    ' fukwhit ',
    ' fukwit ',
    ' fux ',
    ' fux0r ',
    ' f_u_c_k ',
    ' gangbang ',
    ' gangbanged ',
    ' gangbangs ',
    ' gaylord ',
    ' gaysex ',
    ' goatse ',
    ' God ',
    ' god-dam ',
    ' god-damned ',
    ' goddamn ',
    ' goddamned ',
    ' hardcoresex ',
    ' hell ',
    ' heshe ',
    ' hoar ',
    ' hoare ',
    ' hoer ',
    ' homo ',
    ' hore ',
    ' horniest ',
    ' horny ',
    ' hotsex ',
    ' jack-off ',
    ' jackoff ',
    ' jap ',
    ' jerk-off ',
    ' jism ',
    ' jiz ',
    ' jizm ',
    ' jizz ',
    ' kawk ',
    ' knob ',
    ' knobead ',
    ' knobed ',
    ' knobend ',
    ' knobhead ',
    ' knobjocky ',
    ' knobjokey ',
    ' kock ',
    ' kondum ',
    ' kondums ',
    ' kum ',
    ' kummer ',
    ' kumming ',
    ' kums ',
    ' kunilingus ',
    ' l3itch ',
    ' labia ',
    ' lmfao ',
    ' lust ',
    ' lusting ',
    ' m0f0 ',
    ' m0fo ',
    ' m45terbate ',
    ' ma5terb8 ',
    ' ma5terbate ',
    ' masochist ',
    ' master-bate ',
    ' masterb8 ',
    ' masterbat3 ',
    ' masterbate ',
    ' masterbation ',
    ' masterbations ',
    ' masturbate ',
    ' mo-fo ',
    ' mof0 ',
    ' mofo ',
    ' mothafuck ',
    ' mothafucka ',
    ' mothafuckas ',
    ' mothafuckaz ',
    ' mothafucked ',
    ' mothafucker ',
    ' mothafuckers ',
    ' mothafuckin ',
    ' mothafucking ',
    ' mothafuckings ',
    ' mothafucks ',
    ' mother fucker ',
    ' motherfuck ',
    ' motherfucked ',
    ' motherfucker ',
    ' motherfuckers ',
    ' motherfuckin ',
    ' motherfucking ',
    ' motherfuckings ',
    ' motherfuckka ',
    ' motherfucks ',
    ' muff ',
    ' mutha ',
    ' muthafecker ',
    ' muthafuckker ',
    ' muther ',
    ' mutherfucker ',
    ' n1gga ',
    ' n1gger ',
    ' nazi ',
    ' nigg3r ',
    ' nigg4h ',
    ' nigga ',
    ' niggah ',
    ' niggas ',
    ' niggaz ',
    ' nigger ',
    ' niggers ',
    ' nob ',
    ' nob jokey ',
    ' nobhead ',
    ' nobjocky ',
    ' nobjokey ',
    ' numbnuts ',
    ' nutsack ',
    ' orgasim ',
    ' orgasims ',
    ' orgasm ',
    ' orgasms ',
    ' p0rn ',
    ' pawn ',
    ' pecker ',
    ' penis ',
    ' penisfucker ',
    ' phonesex ',
    ' phuck ',
    ' phuk ',
    ' phuked ',
    ' phuking ',
    ' phukked ',
    ' phukking ',
    ' phuks ',
    ' phuq ',
    ' pigfucker ',
    ' pimpis ',
    ' piss ',
    ' pissed ',
    ' pisser ',
    ' pissers ',
    ' pisses ',
    ' pissflaps ',
    ' pissin ',
    ' pissing ',
    ' pissoff ',
    ' poop ',
    ' porn ',
    ' porno ',
    ' pornography ',
    ' pornos ',
    ' prick ',
    ' pricks ',
    ' pron ',
    ' pube ',
    ' pusse ',
    ' pussi ',
    ' pussies ',
    ' pussy ',
    ' pussys ',
    ' rectum ',
    ' retard ',
    ' rimjaw ',
    ' rimming ',
    ' s hit ',
    ' s.o.b. ',
    ' sadist ',
    ' schlong ',
    ' screwing ',
    ' scroat ',
    ' scrote ',
    ' scrotum ',
    ' semen ',
    ' sex ',
    ' sh!t ',
    ' sh1t ',
    ' shag ',
    ' shagger ',
    ' shaggin ',
    ' shagging ',
    ' shemale ',
    ' shit ',
    ' shitdick ',
    ' shite ',
    ' shited ',
    ' shitey ',
    ' shitfuck ',
    ' shitfull ',
    ' shithead ',
    ' shiting ',
    ' shitings ',
    ' shits ',
    ' shitted ',
    ' shitter ',
    ' shitters ',
    ' shitting ',
    ' shittings ',
    ' shitty ',
    ' skank ',
    ' slut ',
    ' sluts ',
    ' smegma ',
    ' smut ',
    ' snatch ',
    ' son-of-a-bitch ',
    ' spac ',
    ' spunk ',
    ' s_h_i_t ',
    ' t1tt1e5 ',
    ' t1tties ',
    ' teets ',
    ' teez ',
    ' testical ',
    ' testicle ',
    ' tit ',
    ' titfuck ',
    ' tits ',
    ' titt ',
    ' tittie5 ',
    ' tittiefucker ',
    ' titties ',
    ' tittyfuck ',
    ' tittywank ',
    ' titwank ',
    ' tosser ',
    ' turd ',
    ' tw4t ',
    ' twat ',
    ' twathead ',
    ' twatty ',
    ' twunt ',
    ' twunter ',
    ' v14gra ',
    ' v1gra ',
    ' vagina ',
    ' viagra ',
    ' vulva ',
    ' w00se ',
    ' wang ',
    ' wank ',
    ' wanker ',
    ' wanky ',
    ' whoar ',
    ' whore ',
    ' willies ',
    ' willy ',
    ' xrated ',
    ' xxx '    
]

In [0]:
# check not in the embedding
replace_with_fuck = []

for swear in swear_words:
    if swear[1:(len(swear)-1)] not in embedding_index:
        replace_with_fuck.append(swear)
        
replace_with_fuck = '|'.join(replace_with_fuck)
replace_with_fuck

' 4r5e | 5h1t | 5hit | ass-fucker | assfucker | assfukka | asswhole | a_s_s | b!tch | b17ch | blow job | boiolas | bollok | boooobs | booooobs | booooooobs | bunny fucker | buttmuch | c0cksucker | carpet muncher | cl1t | cockface | cockmunch | cockmuncher | cocksuka | cocksukka | cokmuncher | coksucka | cunillingus | cuntlick | cuntlicker | cuntlicking | cyalis | cyberfuc | cyberfuck | cyberfucked | cyberfucker | cyberfuckers | cyberfucking | dirsa | dlck | dog-fucker | donkeyribber | ejaculatings | ejakulate | f u c k | f u c k e r | f4nny | faggitt | faggs | fannyflaps | fannyfucker | fanyy | fingerfucker | fingerfuckers | fingerfucks | fistfuck | fistfucked | fistfucker | fistfuckers | fistfucking | fistfuckings | fistfucks | fuckingshitmotherfucker | fuckwhit | fudge packer | fudgepacker | fukwhit | fukwit | fux0r | f_u_c_k | god-dam | kawk | knobead | knobed | knobend | knobjocky | knobjokey | kondum | kondums | kummer | kumming | kums | kunilingus | l3itch | m0f0 | m0fo | m45terb

In [0]:
def handle_swears(text):
    text = re.sub(replace_with_fuck, ' fuck ', text)
    return text

In [0]:
df['comment_text'] = df['comment_text'].apply(lambda x: handle_swears(x))
gc.collect()

87

# preprocess Done

# Train test split 

In [0]:
train = df.iloc[:1804874,:]
test = df.iloc[1804874:,:]

train.head()

,id,comment_text
0,59848,"this is so cool . it is like , ' would yo..."
1,59849,thank you ! ! this would make my life a lot ...
2,59852,this is such an urgent design problem ; kudos...
3,59855,is this something i will be able to install on...
4,59856,haha you guys are a bunch of losers .


In [0]:
del(df)
gc.collect()

11

In [0]:
train_orig = pd.read_csv("/content/train.csv")
train_orig.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,black,buddhist,christian,female,heterosexual,hindu,homosexual_gay_or_lesbian,intellectual_or_learning_disability,jewish,latino,male,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:41.987077+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:42.870083+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:45.222647+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:47.601894+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-29 10:50:48.488476+00,2,NaN,2006,rejected,0,0,0,1,0,0.0,4,47


In [0]:
train = pd.concat([train,train_orig[['target']]],axis=1)
train.head()

,id,comment_text,target
0,59848,"this is so cool . it is like , ' would yo...",0.000000
1,59849,thank you ! ! this would make my life a lot ...,0.000000
2,59852,this is such an urgent design problem ; kudos...,0.000000
3,59855,is this something i will be able to install on...,0.000000
4,59856,haha you guys are a bunch of losers .,0.893617


In [0]:
del(train_orig)
gc.collect()

140

In [0]:
train['target'] = np.where(train['target'] >= 0.5, True, False)

In [0]:
train.head()

,id,comment_text,target
0,59848,"this is so cool . it is like , ' would yo...",False
1,59849,thank you ! ! this would make my life a lot ...,False
2,59852,this is such an urgent design problem ; kudos...,False
3,59855,is this something i will be able to install on...,False
4,59856,haha you guys are a bunch of losers .,True


In [0]:
train_df, validate_df = model_selection.train_test_split(train, test_size=0.1)

print('%d train comments, %d validate comments' % (len(train_df), len(validate_df)))

1624386 train comments, 180488 validate comments


In [0]:
train_df.head()

,id,comment_text,target
1237498,5626929,toronto real estate market is like poker game ...,False
1240517,5630425,really ruben dear where does it say that ? wh...,False
205519,493266,yup . get after it . nothing in there about ...,False
428725,768404,so much sadness .,False
520255,880328,with a little checking i bet this was not the ...,True


In [0]:
MAX_NUM_WORDS = 100000
TOXICITY_COLUMN = 'target'
TEXT_COLUMN = 'comment_text'

# Create a text tokenizer.
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(train_df[TEXT_COLUMN])

MAX_SEQUENCE_LENGTH = 256
def pad_text(texts, tokenizer):
    return pad_sequences(tokenizer.texts_to_sequences(texts), maxlen=MAX_SEQUENCE_LENGTH)

In [0]:
word_bag = []

for word, index in tokenizer.word_index.items():
  word_bag.append(word)
  

In [0]:
len(word_bag)

300876

# get embedding weight

In [0]:
EMBEDDINGS_DIMENSION = 300

embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, EMBEDDINGS_DIMENSION))

In [0]:
num_words_in_embedding = 0

for word, i in tokenizer.word_index.items():
    if word in embedding_index.vocab:
        embedding_vector = embedding_index[word]
        embedding_matrix[i] = embedding_vector        
        num_words_in_embedding += 1

# embedding matrix (words , embedding dim)

In [0]:
embedding_matrix.shape

(300877, 300)

In [0]:
train_text = pad_text(train_df[TEXT_COLUMN], tokenizer)
train_labels = train_df[TOXICITY_COLUMN]

validate_text = pad_text(validate_df[TEXT_COLUMN], tokenizer)
validate_labels = validate_df[TOXICITY_COLUMN]

In [0]:
# define model
# embedding dim = 300
# max_sequence_lenght = 100000

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')

embedding_layer = Embedding(len(tokenizer.word_index) + 1,     # num of words
                            EMBEDDINGS_DIMENSION,              # embedding dim
                            weights=[embedding_matrix],        # inject embedding weight
                            input_length=MAX_SEQUENCE_LENGTH,  # sequence_length
                            trainable=False)                   # freeze layer


x = embedding_layer(sequence_input)
x = SpatialDropout1D(0.2)(x)
x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)   
x = Conv1D(64, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(x)

avg_pool1 = GlobalAveragePooling1D()(x)
max_pool1 = GlobalMaxPooling1D()(x)     

x = concatenate([avg_pool1, max_pool1])

preds = Dense(1, activation='sigmoid')(x)


model = Model(sequence_input, preds)
model.summary()






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 256, 300)     90263100    input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 256, 300)     0           embedding_1[0][0]                
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 256, 128)     140544      spatia

In [0]:
model.compile(loss='binary_crossentropy',
              optimizer=Adam(),
              metrics=['acc'])


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
# check embedding layer weights 

embedding_layer = model.layers[1]

embedding_layer.get_weights()[0].shape

(300877, 300)

# train model

In [0]:
BATCH_SIZE = 1024
NUM_EPOCHS = 100

model.fit(
    train_text,
    train_labels,
    batch_size=BATCH_SIZE,
    epochs=NUM_EPOCHS,
    validation_data=(validate_text, validate_labels),
    callbacks = [EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)])

Train on 1624386 samples, validate on 180488 samples
Epoch 1/100
1624386/1624386 [==============================] - 519s 319us/step - loss: 0.1458 - acc: 0.9446 - val_loss: 0.1277 - val_acc: 0.9488
Epoch 2/100
1624386/1624386 [==============================] - 512s 315us/step - loss: 0.1265 - acc: 0.9500 - val_loss: 0.1227 - val_acc: 0.9507
Epoch 3/100
1624386/1624386 [==============================] - 512s 315us/step - loss: 0.1215 - acc: 0.9517 - val_loss: 0.1249 - val_acc: 0.9490
Epoch 4/100
1624386/1624386 [==============================] - 517s 319us/step - loss: 0.1185 - acc: 0.9528 - val_loss: 0.1199 - val_acc: 0.9523
Epoch 5/100
1624386/1624386 [==============================] - 514s 317us/step - loss: 0.1162 - acc: 0.9536 - val_loss: 0.1209 - val_acc: 0.9505
Epoch 6/100
1624386/1624386 [==============================] - 521s 321us/step - loss: 0.1144 - acc: 0.9543 - val_loss: 0.1213 - val_acc: 0.9503
Epoch 7/100
1624386/1624386 [==============================] - 522s 321us/ste